In [45]:
pwd

'/Users/danielbyrne/Documents'

In [ ]:
### same dataset , 3 different alignment metrics, time of onset of alignment, within convo alignment, temporal delay 
## onset and offset, sequence pos within speaker and between turns, leading & following model, 

In [1]:
import sqlalchemy
#import dependencies
import pandas as pd
import re
import glob
import spacy
nlp = spacy.load('en_core_web_lg')
##check lg vector shape
nlp.vocab.vectors.shape
from collections import Counter
import re

In [53]:
def clean_text(string):
    clean_string = re.sub('[,;:-]','', string)
    return clean_string


In [65]:
def syntactic_lexical_sim(statement1, statement2):
    statement1 =[x.lower().strip() for x in statement1.split()]
    statement2 =[x.lower().strip() for x in statement2.split()]
#     print(statement1, statement2)
    
    if (len(statement1) or len(statement2)) == 0:
#         print('yup')
        return [0,0,0,0]
    
    else:
    
        shared_lexical_items = 0
        num_lexical_items = len(statement1) + len(statement2)

        for word in statement1:
            if word in statement2:
                shared_lexical_items += 1

        lexical_sim_ratio = (shared_lexical_items/num_lexical_items) * 2

        nlp_statement1 = nlp(' '.join(statement1))

        nlp_statement2 = nlp(' '.join(statement2))

        statement1_pos = [x.pos_ for x in nlp_statement1]

        statement2_pos = [x.pos_ for x in nlp_statement2]

        shared_pos = 0

        num_pos =  len(statement1) + len(statement2_pos)

        for pos in statement1_pos:
            if pos in statement2_pos:
                shared_pos += 1

        syntactic_sim_ratio = (shared_pos/num_pos) * 2 

        return [lexical_sim_ratio, syntactic_sim_ratio, num_lexical_items, statement2_pos]
    

###REMOVE & Store password in env variable

SQLALCHEMY_DATABASE_URI = '#####'

# Test if it works
engine = sqlalchemy.create_engine(SQLALCHEMY_DATABASE_URI, echo=True)
print(engine.table_names())

2020-12-10 15:17:59,584 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-12-10 15:17:59,584 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:17:59,601 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-12-10 15:17:59,601 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:18:00,018 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-12-10 15:18:00,020 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:18:00,056 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-12-10 15:18:00,057 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:18:00,134 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-12-10 15:18:00,134 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:18:00,152 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-12-10 15:18:00,153 INFO sqlalchemy.engine.base.E

In [66]:
subj_ids = [
22,
24,
25,
27,
28,
29,
33,
37,
38,
39,
40,
42,
43,
44,
45,
47,
48,
49,
50,
51,
54,
55,
58,
59,
61,
62,
64,
65,
66,
68,
72,
73,
74,
75,
76,
77,
78,
79,
80,
81,
82,
83,
84,
85,
87,
88,
89,
91,
92,
100,
102,
103,
105,
106,
107,
108,
109,
110,
119,
123,
124,
125,
126,
127,
301
]

session_li = [1,2,3,4,5,6,7,8,9,10,11,12]

def means(x):
    if len(x) == 0:
        return 0
    else:
        return sum(x)/len(x)

def parent_label(x):
    if x != '':
        return 'parent'

def child_label(x):
    if x != '':
        return 'child'

completed = []
table_dfs = []

for sub in subj_ids:
    for sesh in session_li:

        table_df = pd.read_sql(
            f"SELECT * FROM utterances WHERE (subject = {sub} and session = {sesh});" ,
            con=engine

        )
        
        table_df['speaker_label'] = table_df['c_utts_orig'].fillna('').map(lambda x: child_label(x))
        table_df['speaker_label'] = table_df['speaker_label'].fillna('').map(lambda x: 'parent' if x == '' else 'child')
        table_df['speaker_shift'] = table_df['speaker_label'].shift() != table_df['speaker_label']
        table_df = table_df[['speaker_label', 'speaker_shift', 'p_utts_orig', 'c_utts_orig']]
        
        table_df['p_utts_orig'] = table_df['p_utts_orig'].fillna('').map(lambda x: clean_text(x))
        table_df['c_utts_orig'] = table_df['c_utts_orig'].fillna('').map(lambda x: clean_text(x))
        
        table_dfs.append(table_df)
        
        count = 0
        sim_list = []
        lexical_pos_sim = []
        dir_parent_child_sim = []
        dir_child_parent_sim = []
        question_dir_parent_child = []
        question_dir_child_parent = []
        non_question_parent_child = []
        non_question_child_parent = []
        semantic_dict = {}
        mean_list = []
        len_list = []
        
        for i, v in enumerate(list(table_df['speaker_shift'])):

            if (v == True) and (list(table_df['speaker_label'])[i] == 'parent'):
                shift_sim = nlp(list(table_df['p_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['c_utts_orig'].fillna(''))[i - 1]))
                lexical_pos = syntactic_lexical_sim(list(table_df['c_utts_orig'].fillna(''))[i - 1], list(table_df['p_utts_orig'].fillna(''))[i])
                dir_child_parent_sim.append(shift_sim)
                sim_list.append(shift_sim)
                lexical_pos_sim.append(lexical_pos)
                if len(nlp((list(table_df['c_utts_orig'].fillna(''))[i - 1])).text) > 0:
                    if nlp((list(table_df['c_utts_orig'].fillna(''))[i - 1])).text[-1] == '?':
                        shift_sim = nlp(list(table_df['p_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['c_utts_orig'].fillna(''))[i - 1]))
                        question_dir_child_parent.append(shift_sim)
                    else:
                        shift_sim = nlp(list(table_df['p_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['c_utts_orig'].fillna(''))[i - 1]))
                        non_question_child_parent.append(shift_sim)
            if (v == True) and (list(table_df['speaker_label'])[i] == 'child'):
                shift_sim = nlp(list(table_df['c_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['p_utts_orig'].fillna(''))[i - 1]))
                dir_parent_child_sim.append(shift_sim)
                sim_list.append(shift_sim)
                lexical_pos = syntactic_lexical_sim(list(table_df['p_utts_orig'].fillna(''))[i - 1], list(table_df['c_utts_orig'].fillna(''))[i])
                lexical_pos_sim.append(lexical_pos)
                if len(nlp((list(table_df['p_utts_orig'].fillna(''))[i - 1])).text) > 0:
                    if nlp((list(table_df['p_utts_orig'].fillna(''))[i - 1])).text[-1] == '?':
                        shift_sim = nlp(list(table_df['c_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['p_utts_orig'].fillna(''))[i - 1]))
                        question_dir_parent_child.append(shift_sim)
                    else:
                        shift_sim = nlp(list(table_df['c_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['p_utts_orig'].fillna(''))[i - 1]))
                        non_question_parent_child.append(shift_sim)
                        
        lexical_li = [x[0] for x in lexical_pos_sim]
        syntactic_li = [x[1] for x in lexical_pos_sim]


        for x in [sim_list,dir_parent_child_sim,
        dir_child_parent_sim,
        question_dir_parent_child,
        question_dir_child_parent,
        non_question_parent_child,
        non_question_child_parent, 
        lexical_li, syntactic_li]:
            mean_list.append(means(x))



        for x in [sim_list,dir_parent_child_sim,
        dir_child_parent_sim,
        question_dir_parent_child,
        question_dir_child_parent,
        non_question_parent_child,
        non_question_child_parent]:
            len_list.append(len(x))

        dict_vals = [sub] + [sesh] + mean_list + len_list

        completed.append(dict_vals)


df = pd.DataFrame.from_records(completed)
# df.to_csv('semantic_shifts.csv')

2020-12-10 15:18:13,700 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 22 and session = 1);`
2020-12-10 15:18:13,701 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:18:13,727 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 15:18:13,769 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 22 and session = 1);
2020-12-10 15:18:13,770 INFO sqlalchemy.engine.base.OptionEngine {}


/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:136: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:123: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:133: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:146: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


2020-12-10 15:18:20,977 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 22 and session = 2);`
2020-12-10 15:18:20,978 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:18:21,001 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 15:18:21,034 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 22 and session = 2);
2020-12-10 15:18:21,035 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 15:18:52,827 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 22 and session = 3);`
2020-12-10 15:18:52,828 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:18:52,867 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 15:18:52,904 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 22 and session = 3);
2020-12-10 15:18:52,904 INFO sqlalchemy.engine.base.OptionEngine {}


/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:143: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


2020-12-10 15:19:28,407 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 22 and session = 4);`
2020-12-10 15:19:28,408 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 15:19:28,739 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 15:19:28,846 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 22 and session = 4);
2020-12-10 15:19:28,848 INFO sqlalchemy.engine.base.OptionEngine {}


...

2020-12-10 19:48:20,663 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:48:54,938 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 119 and session = 11);`
2020-12-10 19:48:54,939 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:48:54,968 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:48:55,012 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 119 and session = 11);
2020-12-10 19:48:55,014 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:49:23,763 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 119 and session = 12);`
2020-12-10 19:49:23,764 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:49:23,786 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:49:23,826 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 119 and session = 12);
2020-12-10 19:49:23,828 INFO sqlalchemy.engine.base.OptionEngine {}
2020-1

2020-12-10 19:51:32,459 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:51:32,653 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 124 and session = 5);`
2020-12-10 19:51:32,654 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:51:32,677 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:51:32,719 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 124 and session = 5);
2020-12-10 19:51:32,721 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:51:42,306 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 124 and session = 6);`
2020-12-10 19:51:42,307 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:51:42,325 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:51:42,360 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 124 and session = 6);
2020-12-10 19:51:42,361 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10

2020-12-10 19:55:03,076 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:55:08,642 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 125 and session = 11);`
2020-12-10 19:55:08,642 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:55:08,669 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:55:08,713 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 125 and session = 11);
2020-12-10 19:55:08,715 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:55:15,592 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 125 and session = 12);`
2020-12-10 19:55:15,593 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:55:15,618 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:55:15,652 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 125 and session = 12);
2020-12-10 19:55:15,653 INFO sqlalchemy.engine.base.OptionEngine {}
2020-1

2020-12-10 19:57:29,026 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:57:42,511 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 127 and session = 5);`
2020-12-10 19:57:42,512 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:57:42,537 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:57:42,574 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 127 and session = 5);
2020-12-10 19:57:42,575 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 19:58:05,554 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 127 and session = 6);`
2020-12-10 19:58:05,555 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 19:58:05,583 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 19:58:05,621 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 127 and session = 6);
2020-12-10 19:58:05,622 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10

2020-12-10 20:01:41,899 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 20:01:42,073 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 301 and session = 11);`
2020-12-10 20:01:42,074 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 20:01:42,093 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 20:01:42,129 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 301 and session = 11);
2020-12-10 20:01:42,131 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-10 20:01:42,282 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 301 and session = 12);`
2020-12-10 20:01:42,283 INFO sqlalchemy.engine.base.Engine {}
2020-12-10 20:01:42,305 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-10 20:01:42,337 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 301 and session = 12);
2020-12-10 20:01:42,338 INFO sqlalchemy.engine.base.OptionEngine {}


In [8]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,22,1,0.540205,0.520586,0.559615,0.533329,0.610342,0.568679,0.564142,187,93,94,30,3,57,90
1,22,2,0.498977,0.469175,0.528705,0.453534,0.943833,0.540547,0.527943,797,398,399,196,2,181,396
2,22,3,0.640782,0.597005,0.684462,0.575647,0.710564,0.674143,0.685433,893,446,447,274,10,161,436
3,22,4,0.652223,0.623936,0.680510,0.643241,0.670730,0.626571,0.681609,1564,782,782,365,79,404,703
4,22,5,0.633421,0.617570,0.649272,0.621096,0.694820,0.653341,0.644434,1354,677,677,285,65,369,612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,301,8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
776,301,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
777,301,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
778,301,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0


In [71]:
col_names =['subject', 'session', 'overall_sim',
 'dir_parent_child_sim',
        'dir_child_parent_sim',
        'question_dir_parent_child_sim',
        'question_dir_child_parent_sim',
        'non_question_parent_child_sim',
        'non_question_child_parent_sim', 
        'lexical_sim', 
        'syntactic_li',
        'speaker_shift_counts',
         'parent_child_shift_counts',
        'child_parent_shift_counts',
        'question_parent_child_counts',
        'question_child_parent_counts',
        'non_question_parent_child_counts',
        'non_question_child_parent_counts']

In [73]:
df.columns = col_names

df['syntactic_li'].tolist()

[0.529202645245426,
 0.47861583209763864,
 0.546264634771648,
 0.5376956389934611,
 0.5264599699372924,
 0.5064768816387685,
 0.5109365037744806,
 0.48874949489391634,
 0.5391041499636461,
 0.4391988527793628,
 0.47800907571407053,
 0.42673323617048187,
 0.5689064173726748,
 0.5502686894378293,
 0.5058838407027825,
 0.5478170535959478,
 0.5466023495271305,
 0.5557379909859304,
 0.5656944935055916,
 0.451596709697974,
 0.4589298391814056,
 0.5105747740409361,
 0.4734499875541639,
 0.4969157719918627,
 0.13333333333333333,
 0.1918229918229918,
 0.27070397177825456,
 0.5606918107423607,
 0.4705858673107566,
 0.4515213332246348,
 0.5088924325211229,
 0.5055197439543007,
 0.5499970513247651,
 0.5176875496844917,
 0.3106725021010735,
 0.47581186343074994,
 0.0,
 0.23270182675447537,
 0.19628239628239627,
 0.5700299105767989,
 0.6134655407936183,
 0.5898987555990406,
 0.5340817373095691,
 0.5699452878134065,
 0.4874373468424213,
 0.4822778153943478,
 0.5340093619584638,
 0.5288263372494932,
 

In [75]:
df.to_csv('ldp_parent_child_alignment_avgs_1210.csv')

In [25]:
dfs = []
for sub in subj_ids:
    for sesh in [5,6,7]:

        table_df = pd.read_sql(
            f"SELECT * FROM utterances WHERE (subject = {sub} and session = {sesh});" ,
            con=engine

        )
        
        table_df['speaker_label'] = table_df['c_utts_orig'].fillna('').map(lambda x: child_label(x))
        table_df['speaker_label'] = table_df['speaker_label'].fillna('').map(lambda x: 'parent' if x == '' else 'child')
        table_df['speaker_shift'] = table_df['speaker_label'].shift() != table_df['speaker_label']
        table_df = table_df[['speaker_label', 'speaker_shift', 'p_utts_orig', 'c_utts_orig']]
        dfs.append(table_df)

2020-12-09 12:30:22,694 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 22 and session = 5);`
2020-12-09 12:30:22,696 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:22,717 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:22,752 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 22 and session = 5);
2020-12-09 12:30:22,752 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:23,191 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 22 and session = 6);`
2020-12-09 12:30:23,192 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:23,217 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:23,258 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 22 and session = 6);
2020-12-09 12:30:23,259 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:23,699 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM

2020-12-09 12:30:35,094 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:36,476 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 33 and session = 5);`
2020-12-09 12:30:36,477 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:36,525 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:36,639 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 33 and session = 5);
2020-12-09 12:30:36,640 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:37,241 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 33 and session = 6);`
2020-12-09 12:30:37,242 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:37,287 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:37,366 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 33 and session = 6);
2020-12-09 12:30:37,367 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:

2020-12-09 12:30:47,621 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:47,960 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 43 and session = 5);`
2020-12-09 12:30:47,961 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:47,976 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:48,014 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 43 and session = 5);
2020-12-09 12:30:48,015 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:48,408 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 43 and session = 6);`
2020-12-09 12:30:48,409 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:48,425 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:48,460 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 43 and session = 6);
2020-12-09 12:30:48,460 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:

2020-12-09 12:30:55,569 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:56,011 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 50 and session = 5);`
2020-12-09 12:30:56,012 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:56,029 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:56,062 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 50 and session = 5);
2020-12-09 12:30:56,063 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:30:56,357 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 50 and session = 6);`
2020-12-09 12:30:56,358 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:30:56,375 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:30:56,412 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 50 and session = 6);
2020-12-09 12:30:56,413 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:

2020-12-09 12:31:01,621 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:01,854 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 61 and session = 5);`
2020-12-09 12:31:01,855 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:01,872 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:01,910 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 61 and session = 5);
2020-12-09 12:31:01,911 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:02,163 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 61 and session = 6);`
2020-12-09 12:31:02,163 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:02,180 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:02,215 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 61 and session = 6);
2020-12-09 12:31:02,216 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:

...

2020-12-09 12:31:13,516 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:13,749 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 78 and session = 5);`
2020-12-09 12:31:13,750 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:13,765 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:13,798 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 78 and session = 5);
2020-12-09 12:31:13,798 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:14,074 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 78 and session = 6);`
2020-12-09 12:31:14,074 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:14,090 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:14,126 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 78 and session = 6);
2020-12-09 12:31:14,128 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:

2020-12-09 12:31:19,752 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:20,093 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 84 and session = 5);`
2020-12-09 12:31:20,094 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:20,111 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:20,140 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 84 and session = 5);
2020-12-09 12:31:20,141 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:20,371 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 84 and session = 6);`
2020-12-09 12:31:20,372 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:20,388 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:20,426 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 84 and session = 6);
2020-12-09 12:31:20,427 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:

2020-12-09 12:31:25,936 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:26,291 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 92 and session = 5);`
2020-12-09 12:31:26,292 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:26,311 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:26,345 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 92 and session = 5);
2020-12-09 12:31:26,346 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:26,777 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 92 and session = 6);`
2020-12-09 12:31:26,778 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:26,794 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:26,827 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 92 and session = 6);
2020-12-09 12:31:26,828 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:

2020-12-09 12:31:33,741 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:34,161 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 107 and session = 5);`
2020-12-09 12:31:34,162 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:34,179 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:34,215 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 107 and session = 5);
2020-12-09 12:31:34,216 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:34,652 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 107 and session = 6);`
2020-12-09 12:31:34,653 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:34,671 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:34,702 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 107 and session = 6);
2020-12-09 12:31:34,703 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09

2020-12-09 12:31:40,938 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:41,050 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 124 and session = 5);`
2020-12-09 12:31:41,051 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:41,066 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:41,105 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 124 and session = 5);
2020-12-09 12:31:41,106 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09 12:31:41,332 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * FROM utterances WHERE (subject = 124 and session = 6);`
2020-12-09 12:31:41,333 INFO sqlalchemy.engine.base.Engine {}
2020-12-09 12:31:41,349 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-12-09 12:31:41,381 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM utterances WHERE (subject = 124 and session = 6);
2020-12-09 12:31:41,381 INFO sqlalchemy.engine.base.OptionEngine {}
2020-12-09

In [26]:
dfs[0]['speaker_shift'].value_counts()

True     1354
False    1164
Name: speaker_shift, dtype: int64

In [27]:
sim_list = []
lexical_pos_sim = []
dir_parent_child_sim = []
dir_child_parent_sim = []
question_dir_parent_child = []
question_dir_child_parent = []
non_question_parent_child = []
non_question_child_parent = []
table_df = dfs[0]
for i, v in enumerate(list(table_df['speaker_shift'])):

    if (v == True) and (list(table_df['speaker_label'])[i] == 'parent'):
        shift_sim = nlp(list(table_df['p_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['c_utts_orig'].fillna(''))[i - 1]))
        lexical_pos = syntactic_lexical_sim(list(table_df['c_utts_orig'].fillna(''))[i - 1], list(table_df['p_utts_orig'].fillna(''))[i])
        dir_child_parent_sim.append(shift_sim)
        sim_list.append(shift_sim)
        print(lexical_pos)
        lexical_pos_sim.append(lexical_pos)
        if len(nlp((list(table_df['c_utts_orig'].fillna(''))[i - 1])).text) > 0:
            if nlp((list(table_df['c_utts_orig'].fillna(''))[i - 1])).text[-1] == '?':
                shift_sim = nlp(list(table_df['p_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['c_utts_orig'].fillna(''))[i - 1]))
                question_dir_child_parent.append(shift_sim)
            else:
                shift_sim = nlp(list(table_df['p_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['c_utts_orig'].fillna(''))[i - 1]))
                non_question_child_parent.append(shift_sim)
    if (v == True) and (list(table_df['speaker_label'])[i] == 'child'):
        shift_sim = nlp(list(table_df['c_utts_orig'].fillna(''))[i]).similarity(nlp(list(table_df['p_utts_orig'].fillna(''))[i - 1]))
        dir_parent_child_sim.append(shift_sim)
        sim_list.append(shift_sim)
        lexical_pos = syntactic_lexical_sim(list(table_df['p_utts_orig'].fillna(''))[i - 1], list(table_df['c_utts_orig'].fillna(''))[i])
        print(lexical_pos)
        lexical_pos_sim.append(lexical_pos)

[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0.0, 0.0, 3, ['DET', 'AUX', 'DET', 'NOUN', 'PUNCT']]
[0, 0, 0, 0]


/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0.0, 0.0, 3, ['PRON', 'AUX', 'DET', 'PUNCT']]
[0, 0, 0, 0]


/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  del sys.path[0]
/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0.0, 0.0, 4, ['PRON', 'VERB', 'DET', 'PUNCT', 'SYM', 'NOUN', 'NOUN', 'PUNCT']]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 

/Users/danielbyrne/opt/anaconda3/envs/ldp_env/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


[0.0, 0.0, 1, ['NUM', 'PUNCT']]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0.0, 0.0, 5, ['INTJ', 'PUNCT', 'DET', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'PUNCT']]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0

[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0.0, 0.0, 1, ['NOUN', 'PUNCT']]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0

In [31]:
for i, v in enumerate(lexical_pos_sim):
     for x in v:
        if x != 0:
            print(i)

14
14
74
74
144
144
158
158
160
160
200
200
338
338
380
380
424
424
428
428
456
456
500
500
548
548
550
550
608
608
620
620
642
642
644
644
762
762
1078
1078
1200
1200
1284
1284
1304
1304


In [44]:
syntactic_lexical_sim('Are you going to read it to me or am I going to read to you?', 'I going to read to you')

['are', 'you', 'going', 'to', 'read', 'it', 'to', 'me', 'or', 'am', 'i', 'going', 'to', 'read', 'to', 'you?'] ['i', 'going', 'to', 'read', 'to', 'you']


[1.6666666666666667,
 2.1666666666666665,
 6,
 ['PRON', 'VERB', 'PART', 'VERB', 'ADP', 'PRON']]

In [35]:
table_df.iloc[13:15]

,speaker_label,speaker_shift,p_utts_orig,c_utts_orig
13,parent,False,Are you going to read it to me or I'm going to...,None
14,child,True,None,I going to read it to you.


In [43]:
syntactic_lexical_sim('I going to read to you', 'Are you going to read it to me or am I going to read to you?')

['i', 'going', 'to', 'read', 'to', 'you'] ['are', 'you', 'going', 'to', 'read', 'it', 'to', 'me', 'or', 'am', 'i', 'going', 'to', 'read', 'to', 'you?']


[0.375,
 0.35294117647058826,
 16,
 ['AUX',
  'PRON',
  'VERB',
  'PART',
  'VERB',
  'PRON',
  'ADP',
  'PRON',
  'CCONJ',
  'AUX',
  'PRON',
  'VERB',
  'PART',
  'VERB',
  'ADP',
  'PRON',
  'PUNCT']]

In [56]:
syntactic_lexical_sim('I going to read to you', 'I going to read to you now')

['i', 'going', 'to', 'read', 'to', 'you'] ['i', 'going', 'to', 'read', 'to', 'you', 'now']


[0.9230769230769231,
 0.9230769230769231,
 13,
 ['PRON', 'VERB', 'PART', 'VERB', 'ADP', 'PRON', 'ADV']]

In [60]:
sum([x[0] for x in lexical_pos_sim])

0.0

In [61]:
len(lexical_pos_sim)

1354

In [63]:
table_df['p_utts_orig'] = table_df['p_utts_orig'].fillna('').map(lambda x: clean_text(x))
table_df['c_utts_orig'] = table_df['c_utts_orig'].fillna('').map(lambda x: clean_text(x))